In [1]:
import warnings

warnings.filterwarnings("ignore")

import sys

sys.path.append("../")
from src.preprocessing.agg_invoice import agg_invoice_num_mode_no_monthly_weighting


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Preprocessing
from sklearn.preprocessing import (
    MinMaxScaler,
    StandardScaler,
    RobustScaler,
    OneHotEncoder,
    OrdinalEncoder,
)

from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
    confusion_matrix,
    classification_report,
)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder

from matplotlib.ticker import PercentFormatter

plt.rcParams.update(
    {"figure.figsize": (8, 5), "axes.facecolor": "white", "axes.edgecolor": "black"}
)
plt.rcParams["figure.facecolor"] = "w"
pd.plotting.register_matplotlib_converters()
pd.set_option("display.float_format", lambda x: "%.3f" % x)


# Define a plotting style to be used for all plots in this notebook
plt.style.use("tableau-colorblind10")

In [2]:
df_client_train = pd.read_csv("data/train/client_train.csv")
df_invoice_train = pd.read_csv("data/train/invoice_train.csv")

df_client_test = pd.read_csv("data/test/client_test.csv")
df_invoice_test = pd.read_csv("data/test/invoice_test.csv")

In [3]:
display(
    df_invoice_train.info(),
    df_invoice_train.describe(),
    df_invoice_train.shape,
    df_invoice_train.head(15),
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4476749 entries, 0 to 4476748
Data columns (total 16 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   client_id             object
 1   invoice_date          object
 2   tarif_type            int64 
 3   counter_number        int64 
 4   counter_statue        object
 5   counter_code          int64 
 6   reading_remarque      int64 
 7   counter_coefficient   int64 
 8   consommation_level_1  int64 
 9   consommation_level_2  int64 
 10  consommation_level_3  int64 
 11  consommation_level_4  int64 
 12  old_index             int64 
 13  new_index             int64 
 14  months_number         int64 
 15  counter_type          object
dtypes: int64(12), object(4)
memory usage: 546.5+ MB


None

,tarif_type,counter_number,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number
count,4476749.000,4476749.000,4476749.000,4476749.000,4476749.000,4476749.000,4476749.000,4476749.000,4476749.000,4476749.000,4476749.000,4476749.000
mean,20.128,123058699065.183,172.488,7.322,1.003,410.979,109.322,20.306,52.926,17766.998,18349.705,44.831
std,13.473,1657267274261.927,133.887,1.572,0.308,757.308,1220.123,157.424,875.472,40366.926,40953.206,3128.335
min,8.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,11.000,121108.000,5.000,6.000,1.000,79.000,0.000,0.000,0.000,1791.000,2056.000,4.000
50%,11.000,494561.000,203.000,8.000,1.000,274.000,0.000,0.000,0.000,7690.000,8192.000,4.000
75%,40.000,1115161.000,207.000,9.000,1.000,600.000,0.000,0.000,0.000,21660.000,22343.000,4.000
max,45.000,27981145458733.000,600.000,413.000,50.000,999910.000,999073.000,64492.000,547946.000,2800280.000,2870972.000,636624.000


(4476749, 16)

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,ELEC
5,train_Client_0,2017-07-17,11,1335667,0,207,9,1,314,0,0,0,15638,15952,8,ELEC
6,train_Client_0,2018-12-07,11,1335667,0,207,9,1,541,0,0,0,15952,16493,12,ELEC
7,train_Client_0,2019-03-19,11,1335667,0,207,9,1,585,0,0,0,16493,17078,8,ELEC
8,train_Client_0,2011-07-22,11,1335667,0,203,9,1,1200,186,0,0,7770,9156,4,ELEC
9,train_Client_0,2011-11-22,11,1335667,0,203,6,1,1082,0,0,0,9156,10238,4,ELEC


In [4]:
df_invoice_train.counter_number.nunique()

201893

In [5]:
# find client with most invoices

df_invoice_train["client_id"].value_counts().head(20)

client_id
train_Client_116878    439
train_Client_9263      380
train_Client_130245    327
train_Client_59131     317
train_Client_61728     310
train_Client_56894     309
train_Client_50639     308
train_Client_4207      307
train_Client_107082    298
train_Client_23141     296
train_Client_21518     293
train_Client_17705     284
train_Client_119751    281
train_Client_103744    280
train_Client_64678     275
train_Client_26663     275
train_Client_84957     275
train_Client_79926     274
train_Client_14725     272
train_Client_42602     272
Name: count, dtype: int64

In [6]:
# convert invoice_date to datetime format
df_invoice_train["invoice_date"] = pd.to_datetime(df_invoice_train["invoice_date"])

In [7]:
# join both datasets by client_id
df_combined = pd.merge(df_client_train, df_invoice_train, on="client_id", how="inner")
df_combined.head(30)

,disrict,client_id,client_catg,region,creation_date,target,invoice_date,tarif_type,counter_number,counter_statue,...,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,60,train_Client_0,11,101,31/12/1994,0.000,2014-03-24,11,1335667,0,...,8,1,82,0,0,0,14302,14384,4,ELEC
1,60,train_Client_0,11,101,31/12/1994,0.000,2013-03-29,11,1335667,0,...,6,1,1200,184,0,0,12294,13678,4,ELEC
2,60,train_Client_0,11,101,31/12/1994,0.000,2015-03-23,11,1335667,0,...,8,1,123,0,0,0,14624,14747,4,ELEC
3,60,train_Client_0,11,101,31/12/1994,0.000,2015-07-13,11,1335667,0,...,8,1,102,0,0,0,14747,14849,4,ELEC
4,60,train_Client_0,11,101,31/12/1994,0.000,2016-11-17,11,1335667,0,...,9,1,572,0,0,0,15066,15638,12,ELEC
5,60,train_Client_0,11,101,31/12/1994,0.000,2017-07-17,11,1335667,0,...,9,1,314,0,0,0,15638,15952,8,ELEC
6,60,train_Client_0,11,101,31/12/1994,0.000,2018-12-07,11,1335667,0,...,9,1,541,0,0,0,15952,16493,12,ELEC
7,60,train_Client_0,11,101,31/12/1994,0.000,2019-03-19,11,1335667,0,...,9,1,585,0,0,0,16493,17078,8,ELEC
8,60,train_Client_0,11,101,31/12/1994,0.000,2011-07-22,11,1335667,0,...,9,1,1200,186,0,0,7770,9156,4,ELEC
9,60,train_Client_0,11,101,31/12/1994,0.000,2011-11-22,11,1335667,0,...,6,1,1082,0,0,0,9156,10238,4,ELEC


In [8]:
# Getting unique values on the invoice train data
for col in df_combined.columns:
    print(f"{col} - {df_combined[col].nunique()}")

disrict - 4
client_id - 135493
client_catg - 3
region - 25
creation_date - 8088
target - 2
invoice_date - 8275
tarif_type - 17
counter_number - 201893
counter_statue - 16
counter_code - 42
reading_remarque - 8
counter_coefficient - 16
consommation_level_1 - 8295
consommation_level_2 - 12576
consommation_level_3 - 2253
consommation_level_4 - 12075
old_index - 155648
new_index - 157980
months_number - 1370
counter_type - 2


In [9]:
df_combined.isnull().sum()

disrict                 0
client_id               0
client_catg             0
region                  0
creation_date           0
target                  0
invoice_date            0
tarif_type              0
counter_number          0
counter_statue          0
counter_code            0
reading_remarque        0
counter_coefficient     0
consommation_level_1    0
consommation_level_2    0
consommation_level_3    0
consommation_level_4    0
old_index               0
new_index               0
months_number           0
counter_type            0
dtype: int64

In [10]:
# Plot fraud vs non-fraud
frauds = df_combined.groupby(["target"])["client_id"].count()
plt.bar(x=frauds.index, height=frauds.values, tick_label=[0, 1])
plt.title("Fraud dist")
plt.show()

In [11]:
df_agg_invoice_train = agg_invoice_num_mode_no_monthly_weighting(df_invoice_train)
# df_ges_test =agg_invoice_num_mode_no_monthly_weighting(df_invoice_test)

In [12]:
df_agg_invoice_train.head()

,client_id,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,counter_coefficient,months_number,invoice_date,tarif_type,counter_statue,counter_code,reading_remarque,counter_type
0,train_Client_0,352.400,10.571,0.000,0.000,1.000,4.629,2011-09-02 18:30:51.428571392,11.000,0,203.000,6.000,ELEC
1,train_Client_1,557.541,0.000,0.000,0.000,1.000,4.324,2012-03-20 20:45:24.324324352,11.000,0,203.000,6.000,ELEC
2,train_Client_10,798.611,37.889,0.000,0.000,1.000,6.444,2011-06-11 05:20:00.000000000,11.000,0,203.000,6.000,ELEC
3,train_Client_100,1.200,0.000,0.000,0.000,1.000,4.200,2009-01-07 18:00:00.000000000,11.000,0,413.000,6.000,ELEC
4,train_Client_1000,663.714,104.857,117.357,36.714,1.000,3.714,2017-06-04 01:42:51.428571392,11.000,0,207.000,9.000,ELEC


###  One-hot encode categorical

In [13]:
df_agg_invoice_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   client_id             135493 non-null  object        
 1   consommation_level_1  135493 non-null  float64       
 2   consommation_level_2  135493 non-null  float64       
 3   consommation_level_3  135493 non-null  float64       
 4   consommation_level_4  135493 non-null  float64       
 5   counter_coefficient   135493 non-null  float64       
 6   months_number         135493 non-null  float64       
 7   invoice_date          135493 non-null  datetime64[ns]
 8   tarif_type            135493 non-null  float64       
 9   counter_statue        135493 non-null  object        
 10  counter_code          135493 non-null  float64       
 11  reading_remarque      135493 non-null  float64       
 12  counter_type          135493 non-null  object        
dtyp

In [14]:
### dummy variables for agg invoice dataset

# select categorical features
cat_features_invoice = [
    "tarif_type",
    "counter_statue",
    "counter_code",
    "reading_remarque",
    "counter_type",
]

for col in cat_features_invoice:
    df_agg_invoice_train[col] = df_agg_invoice_train[col].astype("category")


df_ges_train_dummies = pd.get_dummies(
    df_agg_invoice_train, columns=cat_features_invoice, drop_first=True
)

# added 64 new columns, total 77 columns

In [15]:
df_client_train

,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.000
1,69,train_Client_1,11,107,29/05/2002,0.000
2,62,train_Client_10,11,301,13/03/1986,0.000
3,69,train_Client_100,11,105,11/07/1996,0.000
4,62,train_Client_1000,11,303,14/10/2014,0.000
...,...,...,...,...,...,...
135488,62,train_Client_99995,11,304,26/07/2004,0.000
135489,63,train_Client_99996,11,311,25/10/2012,0.000
135490,63,train_Client_99997,11,311,22/11/2011,0.000
135491,60,train_Client_99998,11,101,22/12/1993,0.000


In [16]:
### dummy variables for client dataset

# select categorical features
cat_features_client = ["disrict", "client_catg", "region"]

for col in cat_features_client:
    df_client_train[col] = df_client_train[col].astype("category")

df_client_train_dummies = pd.get_dummies(
    df_client_train, columns=cat_features_client, drop_first=True
)

# added 26 new columns, total 32 column

df_client_train_dummies

,client_id,creation_date,target,disrict_62,disrict_63,disrict_69,client_catg_12,client_catg_51,region_103,region_104,...,region_308,region_309,region_310,region_311,region_312,region_313,region_371,region_372,region_379,region_399
0,train_Client_0,31/12/1994,0.000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,train_Client_1,29/05/2002,0.000,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,train_Client_10,13/03/1986,0.000,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,train_Client_100,11/07/1996,0.000,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,train_Client_1000,14/10/2014,0.000,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135488,train_Client_99995,26/07/2004,0.000,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
135489,train_Client_99996,25/10/2012,0.000,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
135490,train_Client_99997,22/11/2011,0.000,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
135491,train_Client_99998,22/12/1993,0.000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### handle outliers

In [32]:
df_outliers = df_agg_invoice_train

outlier_col = [
    "consommation_level_1",
    "consommation_level_2",
    "consommation_level_3",
    "consommation_level_4",
]



for col in outlier_col:
    threshold = 3 * df_outliers[col].std()
    mask_outliers = df_outliers[col] > df_outliers[col].mean() + threshold
    df_no_outliers = df_outliers[~mask_outliers]

display(df_no_outliers.describe(),df_agg_invoice_train.describe())

,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,counter_coefficient,months_number,invoice_date
count,134650.000,134650.000,134650.000,134650.000,134650.000,134650.000,134650
mean,430.213,108.924,24.187,31.004,1.002,33.169,2012-11-21 06:44:03.372524032
min,0.000,0.000,0.000,0.000,0.889,1.000,1977-06-16 00:00:00
25%,214.466,0.000,0.000,0.000,1.000,4.000,2012-01-17 09:16:56.470588160
50%,354.835,4.715,0.000,0.000,1.000,4.200,2013-02-24 05:15:40.540540416
75%,539.182,61.532,3.357,0.000,1.000,4.657,2015-10-09 21:36:00
max,99920.000,115683.000,38526.857,2513.133,50.000,602244.750,2019-12-07 00:00:00
std,757.762,728.975,142.637,177.778,0.224,2753.658,NaN


,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,counter_coefficient,months_number,invoice_date
count,135493.000,135493.000,135493.000,135493.000,135493.000,135493.000,135493
mean,432.366,115.334,27.634,73.098,1.002,33.015,2012-11-29 22:29:12.407392
min,0.000,0.000,0.000,0.000,0.889,1.000,1977-06-16 00:00:00
25%,215.281,0.000,0.000,0.000,1.000,4.000,2012-01-20 10:40:00
50%,355.641,4.987,0.000,0.000,1.000,4.200,2013-03-02 21:10:35.294117632
75%,541.889,63.636,3.930,0.000,1.000,4.657,2015-10-25 11:40:32.432432384
max,99920.000,115683.000,38526.857,79179.778,50.000,602244.750,2019-12-07 00:00:00
std,758.004,747.417,157.295,814.217,0.223,2745.088,NaN
